In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('customer_objects.csv', header=0, sep='\t')[["id", "customer_id", "object", "object_type", "object_status", "count", "object_alias", "topic_id_blacklist"]]

In [ ]:
ids = df['id'].values.tolist()
customer_ids = df['customer_id'].values.tolist()
objects = df['object'].values.tolist()
objects_type = df['object_type'].values.tolist()
objects_status = df['object_status'].values.tolist()
counts = df['count'].values.tolist()
objects_alias = df['object_alias'].values.tolist()
topics_id_blacklist = df['topic_id_blacklist'].values.tolist()

In [ ]:
import psycopg2
conn = psycopg2.connect("host= dbname= user= port= password= ")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
def change_object(x):
    y = x.split(' ')
    if len(y) == 2:
        if y[1] == 'район' or y[1] == 'область':
            return x.capitalize()
        if y[0] == 'город':
            return y[1]
    else:
        return x

In [ ]:
def prepare_array(x):
    if x == '[]':
        return '{}'
    else:
        return '{' + str(x) + '}'

In [ ]:
i = 0
while i<len(objects):
    cur.execute("SELECT * FROM customer_objects WHERE object='{0}'".format(objects[i]))
    list_from_db = cur.fetchall()
    df2 = pd.DataFrame(list_from_db, columns=["id", "customer_id", "object", "object_type", "object_status", "count", "object_alias", "topic_id_blacklist"])
    if len(df2) == 0:
        cur.execute("INSERT INTO customer_objects (customer_id, object, object_type, object_status, count, object_alias, topic_id_blacklist) VALUES (0, '{0}', {1}, {2}, 0, '{3}', '{4}');".format(objects[i], objects_type[i], objects_status[i], prepare_array(objects_alias[i]), prepare_array(topics_id_blacklist[i])))
    else:
        alias = df2['object_alias'].values.tolist()[0]
        if objects_alias[i] == '[]':
            new_array = []
        else:
            new_array = objects_alias[i].split(',')
        alias = alias + new_array
        topics = df2['topic_id_blacklist'].values.tolist()[0]
        if topics_id_blacklist[i] == '[]':
            new_array = []
        else:
            new_array = topics_id_blacklist[i].split(',')
        topics = topics + new_array       
        for elem in range(len(alias)):
            alias[elem] = str(alias[elem])
        alias = set(alias)
        alias = list(alias)
        alias = ','.join(alias)
        for elem in range(len(topics)):
            topics[elem] = str(topics[elem])
        topics = set(topics)
        topics = list(topics)
        topics = ','.join(topics)
        cur.execute("UPDATE customer_objects SET object_type={1}, object_status={2}, count=0, object_alias='{3}', topic_id_blacklist='{4}' WHERE object='{0}'".format(objects[i], objects_type[i], objects_status[i], prepare_array(alias), prepare_array(topics)))
    i=i+1

In [ ]:
cur.execute("SELECT * FROM customer_objects")
list_from_db = cur.fetchall()
df3 = pd.DataFrame(list_from_db, columns=["id", "customer_id", "object", "object_type", "object_status", "count", "object_alias", "topic_id_blacklist"])

In [ ]:
objects = df3['object'].values.tolist()

In [ ]:
alias = df3['object_alias'].values.tolist()

In [ ]:
objects_alias = []
for row in alias:
    for elem in row:
        objects_alias.append(elem)
objects_alias = set(objects_alias)

In [ ]:
for elem in objects:
    if elem in objects_alias:
        cur.execute("UPDATE customer_objects SET object_status=2, count=0 WHERE object='{0}'".format(elem))

In [ ]:
cur.close()
conn.close()